# Lab Manual: Build a Simple Knowledge Graph using Python

This lab guides you through the process of building a simple Knowledge Graph (KG) from unstructured text. You will learn how to extract information and then visualize it using two different Python libraries:

- NetworkX: For creating static, publication-quality graph images.

- PyVis: For creating dynamic, interactive HTML-based graphs for exploration.


### Learning Objectives

After completing this lab, you will be able to:

- Understand the basic components of a Knowledge Graph (nodes and edges).

- Use spaCy for basic text processing.

- Extract entities and relationships from text using simple rule-based methods.

- Construct a graph object using NetworkX.

- Generate a static visualization of a KG with NetworkX and Matplotlib.

- Construct a graph object using PyVis.

- Generate a dynamic, interactive visualization of a KG with PyVis.

- Compare the use cases for static vs. dynamic visualizations.

- Understand the foundational concept of how KGs enable reasoning.

### Setup and Imports

In [1]:
import networkx as nx
import matplotlib.pyplot as plt
from pyvis.network import Network
import spacy
import re
from collections import defaultdict

print("✓ All libraries imported successfully!")

# Load spaCy model for NLP
try:
    nlp = spacy.load("en_core_web_sm")
    print("✓ spaCy model loaded")
except:
    print("Please install spaCy model: python -m spacy download en_core_web_sm")

✓ All libraries imported successfully!
✓ spaCy model loaded


### Sample Text and Data Preparation

SAMPLE TEXT FOR KNOWLEDGE GRAPH CONSTRUCTION

We'll use a paragraph about researchers at UCU to build our knowledge graph.

In [2]:
sample_text = """
Dr. Sarah Kizito teaches Computer Science at Uganda Christian University in Mukono. 
She researches Artificial Intelligence and collaborates with Dr. John Mugisha who works on Data Science. 
Dr. Mugisha also teaches Machine Learning courses. Both researchers have published papers on Educational Technology.
UCU offers degrees in Software Engineering and Information Systems. The university is located in Mukono District.
"""

print("Sample Text:")
print("=" * 50)
print(sample_text)

Sample Text:

Dr. Sarah Kizito teaches Computer Science at Uganda Christian University in Mukono. 
She researches Artificial Intelligence and collaborates with Dr. John Mugisha who works on Data Science. 
Dr. Mugisha also teaches Machine Learning courses. Both researchers have published papers on Educational Technology.
UCU offers degrees in Software Engineering and Information Systems. The university is located in Mukono District.



In [3]:
# Process text with spaCy
doc = nlp(sample_text)

print(f"\n✓ Processed {len(doc)} tokens with spaCy")


✓ Processed 69 tokens with spaCy


### Simple Entity and Relationship Extraction

A simple rule-based approach to extract entities and relationships. <br>
In real applications, you would use more sophisticated NLP techniques.

In [6]:
def extract_knowledge_graph(text):
    doc = nlp(text)
    
    # Initialize our graph data
    entities = set()
    relationships = []
    
    # Simple pattern matching for demonstration
    # In a real project, you'd use spaCy's NER, Matcher, or ML models
    patterns = [
        (r'(\w+\s\w+) teaches (\w+(?:\s\w+)*) at (\w+(?:\s\w+)*)', 'teaches_at'),
        (r'(\w+\s\w+) researches (\w+(?:\s\w+)*)', 'researches'),
        (r'(\w+\s\w+) collaborates with (\w+\s\w+)', 'collaborates_with'),
        (r'(\w+(?:\s\w+)*) offers (\w+(?:\s\w+)*)', 'offers'),
        (r'(\w+(?:\s\w+)*) is located in (\w+(?:\s\w+)*)', 'located_in'),
    ]
    
    for pattern, relation_type in patterns:
        matches = re.finditer(pattern, text)
        for match in matches:
            if relation_type == 'teaches_at':
                # This pattern is (Subj) teaches (Topic) at (Obj)
                subject, obj = match.group(1), match.group(3)
            else:
                # All other patterns are (Subj) [verb] (Obj)
                subject, obj = match.group(1), match.group(2)
                
            entities.add(subject)
            entities.add(obj)
            relationships.append((subject, relation_type, obj))
    
    return list(entities), relationships

In [7]:
# Extract knowledge from our sample text
entities, relationships = extract_knowledge_graph(sample_text)

print("EXTRACTED ENTITIES:")
print("-" * 30)
for entity in entities:
    print(f"• {entity}")

print("\nEXTRACTED RELATIONSHIPS:")
print("-" * 30)
for rel in relationships:
    print(f"• {rel[0]} --[{rel[1]}]--> {rel[2]}")

EXTRACTED ENTITIES:
------------------------------
• degrees in Software Engineering and Information Systems
• UCU
• Mukono District
• Uganda Christian University in Mukono
• Sarah Kizito
• The university

EXTRACTED RELATIONSHIPS:
------------------------------
• Sarah Kizito --[teaches_at]--> Uganda Christian University in Mukono
• UCU --[offers]--> degrees in Software Engineering and Information Systems
• The university --[located_in]--> Mukono District


### Static Visualization with NetworkX

NetworkX creates fixed, publication-quality visualizations perfect for analysis.

✓ Graph created with 6 nodes and 3 edges


In [8]:
# Create a directed graph
G = nx.DiGraph()

# Add nodes (entities)
for entity in entities:
    G.add_node(entity, label=entity)

# Add edges (relationships)
for subj, pred, obj in relationships:
    G.add_edge(subj, obj, label=pred, relationship=pred)

print(f"✓ Graph created with {G.number_of_nodes()} nodes and {G.number_of_edges()} edges")

# Visualization settings
plt.figure(figsize=(14, 10))
pos = nx.spring_layout(G, k=3, iterations=50)  # Layout algorithm

# Draw the graph
nx.draw_networkx_nodes(G, pos, node_color='lightblue', 
                      node_size=2000, alpha=0.9)
nx.draw_networkx_edges(G, pos, edge_color='gray', 
                      arrows=True, arrowsize=20, alpha=0.7)
nx.draw_networkx_labels(G, pos, font_size=8, font_weight='bold')

# Add edge labels
edge_labels = nx.get_edge_attributes(G, 'label')
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, 
                           font_size=7, font_color='red')

plt.title("UCU Research Knowledge Graph - Static Visualization (NetworkX)", 
          fontsize=14, fontweight='bold')
plt.axis('off')  # Turn off axis
plt.tight_layout()
plt.show()

In [9]:
# Additional analysis
print("\nGRAPH ANALYSIS:")
print(f"- Nodes: {list(G.nodes())}")
print(f"- Edges: {list(G.edges(data=True))}")
print(f"- Is connected: {nx.is_weakly_connected(G)}")
print(f"- Number of connected components: {nx.number_weakly_connected_components(G)}")


GRAPH ANALYSIS:
- Nodes: ['degrees in Software Engineering and Information Systems', 'UCU', 'Mukono District', 'Uganda Christian University in Mukono', 'Sarah Kizito', 'The university']
- Edges: [('UCU', 'degrees in Software Engineering and Information Systems', {'label': 'offers', 'relationship': 'offers'}), ('Sarah Kizito', 'Uganda Christian University in Mukono', {'label': 'teaches_at', 'relationship': 'teaches_at'}), ('The university', 'Mukono District', {'label': 'located_in', 'relationship': 'located_in'})]
- Is connected: False
- Number of connected components: 3


### Dynamic Visualization with PyVis

In [10]:
# CELL 5: Dynamic Visualization with PyVis
"""
DYNAMIC VISUALIZATION WITH PYYIS
---------------------------------
PyVis creates interactive HTML visualizations perfect for exploration.
"""

# Create a PyVis network
net = Network(height="750px", width="100%", bgcolor="#ffffff", font_color="black",
              directed=True, notebook=True)

# Configure physics for better layout
net.set_options("""
var options = {
  "physics": {
    "enabled": true,
    "stabilization": {"iterations": 100},
    "forceAtlas2Based": {
      "gravitationalConstant": -50,
      "centralGravity": 0.01,
      "springLength": 100,
      "springConstant": 0.08
    }
  },
  "interaction": {"multiselect": true, "navigationButtons": true}
}
""")

# Add nodes with different colors based on type
person_nodes = ['Dr. Sarah Kizito', 'Dr. John Mugisha']
course_nodes = ['Computer Science', 'Data Science', 'Machine Learning', 
                'Software Engineering', 'Information Systems']
location_nodes = ['Mukono', 'Mukono District']
org_nodes = ['Uganda Christian University']
topic_nodes = ['Artificial Intelligence', 'Educational Technology']

node_groups = {
    'Person': person_nodes,
    'Course': course_nodes, 
    'Location': location_nodes,
    'Organization': org_nodes,
    'Research Topic': topic_nodes
}

colors = {
    'Person': '#ff9999',       # Light red
    'Course': '#99ff99',       # Light green
    'Location': '#9999ff',     # Light blue
    'Organization': '#ffff99', # Light yellow
    'Research Topic': '#ff99ff'  # Light magenta
}

# Add nodes to the network
for group, nodes in node_groups.items():
    for node in nodes:
        if node in entities:  # Only add if it exists in our entities
            net.add_node(node, label=node, color=colors[group], 
                        group=group, title=group, font={'size': 14})

# Add edges with different styles based on relationship type
for subj, pred, obj in relationships:
    if subj in entities and obj in entities:
        # Customize edge appearance based on relationship type
        edge_properties = {
            'width': 2,
            'arrowStrikethrough': False,
            'smooth': {'type': 'continuous'}
        }
        
        if 'teaches' in pred:
            edge_properties['color'] = '#3366cc' # Blue
            edge_properties['dashes'] = False
        elif 'research' in pred:
            edge_properties['color'] = '#dc3912' # Red
            edge_properties['dashes'] = [5, 5]
        elif 'collaborates' in pred:
            edge_properties['color'] = '#109618' # Green
            edge_properties['width'] = 3
        else:
            edge_properties['color'] = '#666666' # Gray
            
        net.add_edge(subj, obj, label=pred, **edge_properties)

print("✓ Interactive graph configured")
print(f"- Total nodes: {len(net.nodes)}")
print(f"- Total edges: {len(net.edges)}")

# Save and display
try:
    net.show("ucu_knowledge_graph.html")
    print("✓ Interactive graph saved as 'ucu_knowledge_graph.html'")
    print("→ Open this file in your browser to explore the graph interactively!")
except Exception as e:
    print(f"Error saving graph: {e}")

# Display inline in notebook (if running in Jupyter)
from IPython.display import HTML
HTML(net.generate_html())

AssertionError: non existent node 'Sarah Kizito'